In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
import optuna

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/대구 교통사고 피해 예측/data/'
SEED = 42

In [4]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00


In [5]:
!pip install ngboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.7/350.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 12.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=1d07a804c556572ad6a5d8e543ce26428e2bc267527e4e9eb24770728e621c46
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [6]:
# Modeling & Tuning
from sklearn.model_selection import KFold
n_splits=5; seed = 42
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV


In [7]:
train = pd.read_csv(f"{DATA_PATH}train.csv")

train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
accident_df = pd.read_csv(f"{DATA_PATH}countrywide_accident.csv")
DG_cctv = pd.read_csv(f"{DATA_PATH}대구 CCTV 정보.csv", encoding='cp949')
DG_parking = pd.read_csv(f"{DATA_PATH}대구 주차장 정보.csv", encoding='cp949')
DG_children_area = pd.read_csv(f"{DATA_PATH}대구 어린이 보호 구역 정보.csv", encoding='cp949')
DG_seculight = pd.read_csv(f"{DATA_PATH}대구 보안등 정보.csv", encoding='cp949')
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# KR_cctv = pd.read_csv(f'{DATA_PATH}전국무인교통단속카메라표준데이터.csv', encoding='cp949')
# KR_seculight =pd.read_csv(f'{DATA_PATH}전국보안등정보표준데이터.csv', encoding='cp949')
# KR_children_area = pd.read_csv(f'{DATA_PATH}전국어린이보호구역표준데이터.csv', encoding='cp949')
# KR_parking = pd.read_csv(f'{DATA_PATH}전국주차장정보표준데이터.csv', encoding='cp949')

# train_df.shape, test_df.shape

# DATA_PATH = '/kaggle/input/daegu-dataset-2/'

# train = pd.read_csv(f"{DATA_PATH}train.csv")

# train_df = pd.read_csv(f"{DATA_PATH}train.csv")
# test_df = pd.read_csv(f"{DATA_PATH}test.csv")
# accident_df = pd.read_csv(f"{DATA_PATH}countrywide_accident.csv")
# DG_cctv = pd.read_csv(f"{DATA_PATH}DG_cctv.csv", encoding='cp949')
# DG_parking = pd.read_csv(f"{DATA_PATH}DG_parking.csv", encoding='cp949')
# DG_children_area = pd.read_csv(f"{DATA_PATH}DG_children_area.csv", encoding='cp949')
# DG_seculight = pd.read_csv(f"{DATA_PATH}DG_seculight.csv", encoding='cp949')
# submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

# DATA_PATH = '/kaggle/input/daegu-kr-dataset/'

# KR_cctv = pd.read_csv(f'{DATA_PATH}KR_cctv.csv', encoding='cp949')
# KR_seculight =pd.read_csv(f'{DATA_PATH}KR_seculight.csv', encoding='cp949')
# KR_children_area = pd.read_csv(f'{DATA_PATH}KR_children_area.csv', encoding='cp949')
# KR_parking = pd.read_csv(f'{DATA_PATH}KR_parking.csv', encoding='cp949')

train_df.shape, test_df.shape

((39609, 23), (10963, 8))

In [8]:
train_df = pd.concat([train, accident_df], axis=0).reset_index().drop(columns='index')

In [9]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_df['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

# test 데이터 반영
test_df[['도시', '구', '동']] = test_df['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

In [10]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_df['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

# test 데이터 반영
test_df[['도로형태1', '도로형태2']] = test_df['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

In [11]:
# 동별 연별 총부상수의 평균
# 연별 치사율의 평균

In [12]:
# train_df['도시'].unique().tolist()

# drop_region = ['서울특별시','제주특별자치도','세종특별자치시','미분류','nan']

# mask = ~train_df['도시'].isin(drop_region)
# train_df = train_df[mask].reset_index().drop(columns='index')


# drop_region = ['서울특별시','제주특별자치도','세종특별자치시','미분류','nan']
gwangyuk = ['대구광역시','인천광역시','광주광역시','부산광역시','울산광역시','대전광역시']

mask = train_df['도시'].isin(gwangyuk)
train_df = train_df[mask].reset_index().drop(columns='index')

In [13]:
train_df.columns.tolist()

['ID',
 '사고일시',
 '요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '사고유형 - 세부분류',
 '법규위반',
 '가해운전자 차종',
 '가해운전자 성별',
 '가해운전자 연령',
 '가해운전자 상해정도',
 '피해운전자 차종',
 '피해운전자 성별',
 '피해운전자 연령',
 '피해운전자 상해정도',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상자수',
 'ECLO',
 '도시',
 '구',
 '동',
 '도로형태1',
 '도로형태2']

# Feature 실험

### 시간추가

In [14]:
import holidays
kr_holidays = holidays.KR()
# 시간 관련
train_df['date'] = pd.to_datetime(train_df['사고일시'])
train_df['연'] = train_df['date'].dt.year.astype(int)
train_df['월'] = train_df['date'].dt.month.astype(int)
train_df['일'] = train_df['date'].dt.day.astype(int)
train_df['시간'] = train_df['date'].dt.hour.astype(int)
train_df['요일'] = train_df['date'].dt.day_of_week.astype(int)
train_df['공휴일'] = train_df['date'].apply(lambda x : int(x in kr_holidays)).astype(int)

test_df['date'] = pd.to_datetime(test_df['사고일시'])
test_df['연'] = test_df['date'].dt.year.astype(int)
test_df['월'] = test_df['date'].dt.month.astype(int)
test_df['일'] = test_df['date'].dt.day.astype(int)
test_df['시간'] = test_df['date'].dt.hour.astype(int)
test_df['요일'] = test_df['date'].dt.day_of_week.astype(int)
test_df['공휴일'] = test_df['date'].apply(lambda x : int(x in kr_holidays)).astype(int)

train_df = train_df.drop(columns=['사고일시','date'])
test_df = test_df.drop(columns=['사고일시','date'])

### 총부상자

In [15]:
train_df['월동_총부상자'] = train_df['사망자수'] + train_df['중상자수'] + train_df['경상자수'] + train_df['부상자수']

casualty = train_df.groupby(['월','도시','구','동'])[['월동_총부상자']].mean().reset_index()
train_df = train_df.drop(columns = '월동_총부상자')

train_df = pd.merge(train_df,casualty, on = ['월','도시','구','동'], how='left')
test_df = pd.merge(test_df,casualty, on = ['월','도시','구','동'], how='left')



### eclo mean

In [16]:
tmp = train_df.groupby(['시간','요일'])[['ECLO']].mean().reset_index()
tmp.columns = ['시간','요일','ECLO_mean']

train_df = train_df.merge(tmp, on= ['시간','요일'], how='left')
test_df = test_df.merge(tmp, on= ['시간','요일'], how='left')

### train only

In [17]:
# train_ft1 = pd.concat([train,accident_df], axis= 0).reset_index()

train_ft1 = train

In [18]:
# 시간
train_ft1['date'] = pd.to_datetime(train_ft1['사고일시'])
train_ft1['연'] = train_ft1['date'].dt.year.astype(int)
train_ft1['월'] = train_ft1['date'].dt.month.astype(int)
train_ft1['일'] = train_ft1['date'].dt.day.astype(int)
train_ft1['시간'] = train_ft1['date'].dt.hour.astype(int)
train_ft1['요일'] = train_ft1['date'].dt.day_of_week.astype(int)

# 시군구
location_pattern = r'(\S+) (\S+) (\S+)'
train_ft1[['도시', '구', '동']] = train_ft1['시군구'].str.extract(location_pattern)
train_ft1 = train_ft1.drop(columns=['시군구'])

# 도로형태

road_pattern = r'(.+) - (.+)'
train_ft1[['도로형태1', '도로형태2']] = train_ft1['도로형태'].str.extract(road_pattern)
train_ft1 = train_ft1.drop(columns=['도로형태'])

In [19]:
def categorize_time_period(hour):
    if 6 <= hour < 9:  # 아침 출퇴근 시간
        return 0
    elif 9 <= hour < 12:  # 아침
        return 1
    elif 12 <= hour < 14:  # 점심
        return 2
    elif 14 <= hour < 18:  # 오후
        return 3
    elif 18 <= hour < 21:  # 저녁
        return 4
    elif 21 <= hour < 24:  # 밤
        return 5
    else:  # 새벽
        return 6

train_ft1['시간대'] = train_ft1['시간'].apply(categorize_time_period)

train_df['시간대'] = train_df['시간'].apply(categorize_time_period)
test_df['시간대'] = test_df['시간'].apply(categorize_time_period)

In [20]:

df = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].mean().reset_index()
df0 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].count().reset_index().iloc[:,1:]
df1 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].sum().reset_index().iloc[:,1:]
df2 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].min().reset_index().iloc[:,1:]
df3 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].max().reset_index().iloc[:,1:]
df4 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].median().reset_index().iloc[:,1:]
df5 = train_ft1.groupby(['가해운전자 차종'])[['ECLO']].std().reset_index().iloc[:,1:]


df_acc = pd.concat([df, df4,df0, df1, df2, df3, df5], axis=1)
df_acc.columns = ['가해운전자 차종','median','mean','count','sum','min','max','std']
# df_acc['max_mean'] = df_acc['max'] * df_acc['mean']
# df_acc['max_median'] = df_acc['max'] * df_acc['median']
# df_acc['max_std'] = df_acc['max'] * df_acc['std']
# df_acc['mean_median'] = df_acc['mean'] * df_acc['median']
# df_acc['max_count'] = df_acc['max'] * (df_acc['count'])
# df_acc['max_count_medianmean'] = df_acc['max'] * df_acc['count'] * (df_acc['mean'] + df_acc['median'])/2
# df_acc['max_count_log'] = df_acc['max'] * np.log(df_acc['count'])
# df_acc['max_count_log_mean_log'] = df_acc['max'] * np.log(df_acc['count']) *np.log(df_acc['mean'])
# df_acc['count_log_median'] = np.log(df_acc['count']) * df_acc['median']
# df_acc['max_count_log_median'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['median']
# df_acc['max_count_log_mean'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['mean']
df_acc['max_std_count_log'] = df_acc['max'] * np.log(df_acc['count']) *df_acc['std']


# max count_log의 조합을 기준으로 정렬해서 라벨 인코딩 (비슷한 범주가 라벨링된다는 가정하에 범주를 평균할 수 있으므로)
df_acc = df_acc.sort_values(by='max_std_count_log', ascending=False)
# display(df_acc)

df_cols = df_acc['가해운전자 차종'].tolist()

label_mapping = {label: index for index, label in enumerate(df_cols)}
train_ft1['가해운전자 차종_sorted'] = train_ft1['가해운전자 차종'].map(label_mapping).astype(int)

# for col in train_ft1[['가해운전자 차종_sorted', '가해운전자 차종']]:
#     sns.scatterplot(x=train_ft1[col], y=train_ft1['ECLO'])
#     break

In [21]:
df = train_ft1.groupby(['구','시간대','요일','사고유형','도로형태2'])[['가해운전자 차종_sorted']].mean().reset_index()
df

,구,시간대,요일,사고유형,도로형태2,가해운전자 차종_sorted
0,남구,0,0,차대사람,교차로부근,1.000000
1,남구,0,0,차대사람,교차로횡단보도내,1.000000
2,남구,0,0,차대사람,기타,3.200000
3,남구,0,0,차대사람,주차장,2.000000
4,남구,0,0,차대차,교차로부근,2.833333
...,...,...,...,...,...,...
3775,중구,6,6,차대차,교량,2.000000
3776,중구,6,6,차대차,교차로부근,2.000000
3777,중구,6,6,차대차,교차로안,2.142857
3778,중구,6,6,차대차,기타,2.304348


In [22]:
train_df = pd.merge(train_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')
test_df = pd.merge(test_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')


In [23]:

df = train_ft1.groupby(['피해운전자 상해정도'])[['ECLO']].mean().reset_index()
df1 = train_ft1.groupby(['피해운전자 상해정도'])[['ECLO']].min().reset_index().iloc[:,1:]

df_acc = pd.concat([df,df1], axis=1)
df_acc.columns = ['피해운전자 상해정도','mean','min']
df_acc = df_acc.sort_values(by='min', ascending=False)

df_cols = df_acc['피해운전자 상해정도'].tolist()

label_mapping = {label: index for index, label in enumerate(df_cols)}
train_ft1['피해운전자 상해정도_sorted'] = train_ft1['피해운전자 상해정도'].map(label_mapping)

# for col in train_ft1[['피해운전자 상해정도_sorted', '피해운전자 상해정도']]:
#     sns.scatterplot(x=train_ft1[col], y=train_ft1['ECLO'])
#     break

In [24]:
df = train_ft1.groupby(['구','시간대','요일','사고유형','도로형태2'])[['피해운전자 상해정도_sorted']].mean().reset_index()
df

,구,시간대,요일,사고유형,도로형태2,피해운전자 상해정도_sorted
0,남구,0,0,차대사람,교차로부근,2.000000
1,남구,0,0,차대사람,교차로횡단보도내,1.000000
2,남구,0,0,차대사람,기타,3.200000
3,남구,0,0,차대사람,주차장,2.000000
4,남구,0,0,차대차,교차로부근,2.500000
...,...,...,...,...,...,...
3775,중구,6,6,차대차,교량,2.000000
3776,중구,6,6,차대차,교차로부근,2.285714
3777,중구,6,6,차대차,교차로안,2.000000
3778,중구,6,6,차대차,기타,2.391304


In [25]:
train_df = pd.merge(train_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')
test_df = pd.merge(test_df,df, on = ['구','시간대','요일','사고유형','도로형태2'], how='left')


In [26]:
train_df = train_df.drop(columns = '시간대')
test_df = test_df.drop(columns = '시간대')

# 인코딩, 스케일링

In [27]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [28]:
train_df.columns.tolist()

['ID',
 '요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '사고유형 - 세부분류',
 '법규위반',
 '가해운전자 차종',
 '가해운전자 성별',
 '가해운전자 연령',
 '가해운전자 상해정도',
 '피해운전자 차종',
 '피해운전자 성별',
 '피해운전자 연령',
 '피해운전자 상해정도',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상자수',
 'ECLO',
 '도시',
 '구',
 '동',
 '도로형태1',
 '도로형태2',
 '연',
 '월',
 '일',
 '시간',
 '공휴일',
 '월동_총부상자',
 'ECLO_mean',
 '가해운전자 차종_sorted',
 '피해운전자 상해정도_sorted']

In [29]:
test_x = test_df.drop(columns = ['ID','도시'])
train_x = train_df[test_x.columns]

train_y = train_df['ECLO']

In [30]:
num_cols = train_x.select_dtypes(exclude='object').columns.tolist()
num_cols

['요일',
 '연',
 '월',
 '일',
 '시간',
 '공휴일',
 '월동_총부상자',
 'ECLO_mean',
 '가해운전자 차종_sorted',
 '피해운전자 상해정도_sorted']

In [31]:
cat_cols = train_x.select_dtypes(include='object').columns.tolist()
cat_cols

['기상상태', '노면상태', '사고유형', '구', '동', '도로형태1', '도로형태2']

In [32]:
cat_cols
# 시간, 공간 정보 이외 모두 one hot encoding

cat_cols1 = ['요일','구','동']
cat_cols2 = [col for col in cat_cols if col not in cat_cols1]
cat_cols2

['기상상태', '노면상태', '사고유형', '도로형태1', '도로형태2']

In [33]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00


In [34]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
import category_encoders as ce

one_hot = OneHotEncoder()
label = LabelEncoder()
ordinal = OrdinalEncoder()
target_enc = ce.TargetEncoder()

In [35]:
train_x[cat_cols1] = train_x[cat_cols1].astype(str)

In [36]:
# cols1 라벨인코딩
for col in cat_cols1 :
    train_x[col] = target_enc.fit_transform(train_x[col],train_y)
    test_x[col] = target_enc.transform(test_x[col])

In [37]:
train_x[cat_cols2] = train_x[cat_cols2].astype(str)

In [38]:
# cols 2 원핫인코딩
tmp = pd.DataFrame(one_hot.fit_transform(train_x[cat_cols2]).toarray())
tmp.columns = one_hot.get_feature_names_out()
pd.DataFrame(tmp)

train_x = pd.concat([train_x,tmp],axis=1).drop(columns= cat_cols2)

tmp = pd.DataFrame(one_hot.fit_transform(test_x[cat_cols2]).toarray())
tmp.columns = one_hot.get_feature_names_out()
pd.DataFrame(tmp)

test_x = pd.concat([test_x,tmp],axis=1).drop(columns= cat_cols2)

train_x.shape, test_x.shape

((158959, 45), (10963, 41))

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
# scaler = StandardScaler()

# train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
# test_df[num_cols] = scaler.transform(test_df[num_cols])

scaler = StandardScaler()

train_x[num_cols] = scaler.fit_transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

# 검증 및 평가

In [42]:
# target = np.log1p(train_y)
# target = train_y

In [43]:
import re
train_x = train_x.rename(columns = lambda x:re.sub('[^가-힣A-Za-z0-9_]+', '', x))
test_x = test_x.rename(columns = lambda x:re.sub('[^가-힣A-Za-z0-9_]+', '', x))

In [44]:
from sklearn.metrics import make_scorer

def rmsle(y_actual, y_pred):
    diff = np.log1p(y_pred) - np.log1p(y_actual)
    mean_error = np.mean(np.square(diff))

    return np.sqrt(mean_error)

rmsle_score = make_scorer(rmsle, greater_is_better=False)

In [45]:
# -0.15797365915856704
lgbm_best_params = {'n_estimators': 1178, 'num_leaves': 30, 'max_depth': 6, 'learning_rate': 0.011407608181541297, 'feature_fraction': 0.8545377839510566, 'bagging_fraction': 0.17429104930042236, 'lambda_l1': 6.529596581208154, 'lambda_l2': 8.837614837937567, 'min_child_samples': 59}

# -0.15789535340990057
cat_best_params = {'n_estimators': 1179, 'depth': 7, 'learning_rate': 0.02164558958549747, 'l2_leaf_reg': 3.0882038843672674}

# -0.4180133168978298
xgb_best_params = {'n_estimators': 3241, 'max_depth': 24, 'learning_rate': 0.024010747652112696, 'gamma': 0.7071727358928532, 'min_child_weight': 2, 'subsample': 0.8062272924485435, 'colsample_bytree': 0.8429061023478192, 'reg_alpha': 0.1812908822349264, 'reg_lambda': 1.7344690669541394}


# 시드앙상블

- xgb seed확인


In [47]:
target = train_y

seed_scores = []

for SEED in tqdm(range(1000)) :
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []
    # models = []
    for tri , vai in cv.split(train_x): # k fold 앙상블
        # 학습 데이터
        x_train = train_x.iloc[tri]
        y_train = train_y.iloc[tri]

        # 검증 데이터
        x_valid = train_x.iloc[vai]
        y_valid = train_y.iloc[vai]

        model = XGBRegressor(random_state=SEED, n_jobs=-1, verbosity = 0)
        model.fit(x_train, y_train)
        pred = model.predict(x_valid)
        score = rmsle(y_valid,pred)
        scores.append(score)

    avg_score = np.mean(scores)
    print(f"SEED {SEED} - RMSLE : {avg_score}")
    seed_scores.append(avg_score)


  0%|          | 0/1000 [00:00<?, ?it/s]

SEED 0 - RMSLE : 0.4355168989540685
SEED 1 - RMSLE : 0.43563630459865543
SEED 2 - RMSLE : 0.43587797814448404
SEED 3 - RMSLE : 0.4356939471790092
SEED 4 - RMSLE : 0.4358951164853999
SEED 5 - RMSLE : 0.4354450363150996
SEED 6 - RMSLE : 0.43608519673566287
SEED 7 - RMSLE : 0.4358704094739
SEED 8 - RMSLE : 0.43578137029562825
SEED 9 - RMSLE : 0.43525937148032073
SEED 10 - RMSLE : 0.435940410636489
SEED 11 - RMSLE : 0.43568737764485743
SEED 12 - RMSLE : 0.43500664057429106
SEED 13 - RMSLE : 0.43591652203307013
SEED 14 - RMSLE : 0.4357967656395824
SEED 15 - RMSLE : 0.4352504419720913
SEED 16 - RMSLE : 0.4357145292666519
SEED 17 - RMSLE : 0.4353609950953806
SEED 18 - RMSLE : 0.43569243994102447
SEED 19 - RMSLE : 0.43562173728811715
SEED 20 - RMSLE : 0.43574646395021305
SEED 21 - RMSLE : 0.43583642463117755
SEED 22 - RMSLE : 0.435430965216165
SEED 23 - RMSLE : 0.43575813091426746
SEED 24 - RMSLE : 0.43569709863215805
SEED 25 - RMSLE : 0.435789981863923
SEED 26 - RMSLE : 0.43549670773605165
SE

In [48]:
seed_ord = []

for idx, score in enumerate(sorted(seed_scores)):
    seed_ord.append(seed_scores.index(score))

print(seed_ord)

[338, 67, 700, 878, 12, 527, 600, 893, 897, 51, 609, 794, 148, 492, 937, 586, 885, 311, 104, 859, 236, 756, 659, 304, 582, 714, 411, 796, 212, 265, 944, 834, 101, 978, 358, 592, 983, 280, 153, 171, 161, 490, 722, 15, 665, 833, 29, 936, 601, 587, 36, 9, 999, 930, 75, 639, 139, 493, 189, 651, 724, 238, 82, 705, 506, 780, 105, 572, 906, 479, 982, 625, 392, 892, 725, 135, 972, 52, 58, 39, 645, 691, 406, 193, 456, 84, 334, 844, 573, 954, 510, 607, 674, 924, 362, 385, 635, 654, 629, 881, 664, 310, 262, 256, 356, 207, 180, 504, 474, 261, 124, 472, 900, 660, 147, 412, 69, 377, 652, 63, 623, 967, 167, 491, 257, 331, 668, 17, 316, 160, 330, 190, 154, 736, 782, 342, 364, 778, 523, 807, 214, 344, 360, 634, 539, 840, 59, 230, 952, 260, 687, 679, 290, 868, 40, 235, 537, 694, 252, 494, 986, 155, 123, 119, 195, 476, 341, 822, 643, 823, 42, 199, 835, 239, 843, 277, 721, 448, 297, 742, 373, 984, 913, 85, 898, 93, 404, 581, 302, 774, 735, 110, 87, 446, 438, 137, 141, 284, 22, 684, 35, 200, 940, 473, 237,

- cat seed확인


In [51]:
target = np.log1p(train_y)

seed_scores = []

for SEED in tqdm(range(1000)) :
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []
    # models = []
    for tri , vai in cv.split(train_x): # k fold 앙상블
        # 학습 데이터
        x_train = train_x.iloc[tri]
        y_train = train_y.iloc[tri]

        # 검증 데이터
        x_valid = train_x.iloc[vai]
        y_valid = train_y.iloc[vai]

        model = CatBoostRegressor(random_state=SEED, verbose = 0)
        model.fit(x_train, y_train)
        pred = model.predict(x_valid)
        score = rmsle(y_valid,pred)
        scores.append(score)

    avg_score = np.mean(scores)
    print(f"SEED {SEED} - RMSLE : {avg_score}")
    seed_scores.append(avg_score)


  0%|          | 0/1000 [00:00<?, ?it/s]

SEED 0 - RMSLE : 0.43181636973061677
SEED 1 - RMSLE : 0.4315134478426728
SEED 2 - RMSLE : 0.43194459779315786
SEED 3 - RMSLE : 0.43193538775642837
SEED 4 - RMSLE : 0.43186493372081786
SEED 5 - RMSLE : 0.43174905043175127
SEED 6 - RMSLE : 0.4319266418425863
SEED 7 - RMSLE : 0.4317416488303876
SEED 8 - RMSLE : 0.4317930489585267
SEED 9 - RMSLE : 0.43154275053356317
SEED 10 - RMSLE : 0.43159876028172206
SEED 11 - RMSLE : 0.43167030811026513
SEED 12 - RMSLE : 0.43183627388937695
SEED 13 - RMSLE : 0.4317985734209441
SEED 14 - RMSLE : 0.4319096216427417
SEED 15 - RMSLE : 0.4317087090333393
SEED 16 - RMSLE : 0.4318715981586518
SEED 17 - RMSLE : 0.43165821529613424
SEED 18 - RMSLE : 0.43161352381382906
SEED 19 - RMSLE : 0.4316797383970254
SEED 20 - RMSLE : 0.4319091907183692
SEED 21 - RMSLE : 0.4318445207453429
SEED 22 - RMSLE : 0.43175313051114894
SEED 23 - RMSLE : 0.43194513960976566
SEED 24 - RMSLE : 0.4315629088960655
SEED 25 - RMSLE : 0.4320159544866594
SEED 26 - RMSLE : 0.431654729360534

KeyboardInterrupt: ignored

In [ ]:
seed_ord = []

for idx, score in enumerate(sorted(seed_scores)):
    seed_ord.append(seed_scores.index(score))

print(seed_ord)

- lgbm seed확인


In [ ]:
target = np.log1p(train_y)

seed_scores = []

for SEED in tqdm(range(1000)) :
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []
    # models = []
    for tri , vai in cv.split(train_x): # k fold 앙상블
        # 학습 데이터
        x_train = train_x.iloc[tri]
        y_train = train_y.iloc[tri]

        # 검증 데이터
        x_valid = train_x.iloc[vai]
        y_valid = train_y.iloc[vai]

        model = LGBMRegressor(random_state=SEED, n_jobs=-1, verbosity = 0)
        model.fit(x_train, y_train)
        pred = model.predict(x_valid)
        score = rmsle(y_valid,pred)
        scores.append(score)

    avg_score = np.mean(scores)
    print(f"SEED {SEED} - RMSLE : {avg_score}")
    seed_scores.append(avg_score)


In [ ]:
seed_ord = []

for idx, score in enumerate(sorted(seed_scores)):
    seed_ord.append(seed_scores.index(score))

print(seed_ord)

In [ ]:
[700, 560, 586, 542, 80, 190, 786, 267, 839, 221, 537, 353, 575, 104, 12, 561, 4, 916, 707, 21, 366, 130, 286, 101, 476, 226, 693, 297, 673, 203, 944, 64, 609, 75, 242, 167, 305, 352, 871, 0, 136, 697, 983, 440, 872, 402, 284, 819, 742, 584, 241, 74, 606, 462, 868, 88, 205, 978, 212, 675, 842, 84, 737, 557, 533, 778, 82, 680, 900, 121, 721, 820, 745, 377, 801, 690, 890, 743, 644, 2, 709, 473, 189, 547, 465, 932, 782, 230, 695, 279, 974, 747, 500, 9, 362, 413, 956, 719, 449, 414, 410, 456, 658, 924, 251, 661, 389, 724, 954, 269, 808, 204, 56, 308, 331, 861, 681, 527, 846, 906, 741, 273, 154, 349, 794, 780, 829, 243, 22, 291, 164, 403, 672, 445, 498, 468, 774, 793, 227, 244, 486, 354, 811, 55, 694, 564, 424, 665, 570, 705, 400, 821, 590, 513, 580, 950, 271, 312, 195, 268, 161, 119, 313, 763, 467, 109, 329, 289, 824, 41, 652, 333, 512, 157, 496, 338, 687, 1, 992, 826, 148, 637, 691, 996, 688, 8, 95, 543, 797, 980, 749, 117, 897, 100, 504, 883, 562, 53, 67, 597, 683, 601, 408, 708, 309, 859, 613, 364, 248, 910, 852, 206, 337, 134, 788, 3, 26, 654, 263, 999, 898, 123, 407, 804, 614, 341, 270, 881, 735, 854, 323, 765, 961, 507, 24, 208, 679, 632, 10, 830, 619, 168, 185, 127, 604, 45, 96, 832, 668, 236, 207, 914, 835, 633, 511, 655, 857, 129, 809, 785, 406, 686, 739, 380, 32, 392, 452, 639, 873, 494, 225, 131, 153, 891, 796, 17, 588, 594, 200, 659, 143, 524, 63, 47, 213, 703, 522, 438, 607, 893, 103, 546, 696, 180, 946, 172, 568, 844, 461, 657, 939, 495, 361, 874, 753, 955, 249, 184, 554, 718, 20, 953, 958, 343, 689, 146, 731, 803, 716, 427, 882, 934, 752, 388, 738, 825, 429, 663, 152, 278, 518, 565, 636, 904, 30, 592, 298, 666, 887, 977, 156, 368, 58, 39, 903, 381, 630, 479, 558, 85, 78, 509, 768, 31, 105, 257, 275, 754, 15, 335, 787, 927, 949, 296, 426, 219, 194, 118, 319, 698, 845, 165, 641, 351, 162, 192, 940, 585, 128, 166, 576, 304, 987, 772, 751, 375, 608, 223, 795, 348, 766, 191, 965, 282, 401, 602, 149, 629, 137, 124, 69, 583, 235, 33, 621, 451, 358, 998, 761, 34, 133, 272, 888, 853, 538, 593, 422, 624, 722, 201, 938, 869, 610, 937, 466, 214, 404, 469, 798, 662, 833, 984, 489, 508, 913, 671, 464, 38, 577, 600, 627, 110, 497, 529, 446, 390, 425, 574, 183, 176, 544, 376, 28, 65, 638, 170, 140, 779, 48, 993, 589, 135, 862, 66, 316, 506, 158, 653, 968, 642, 783, 411, 173, 677, 453, 306, 572, 918, 595, 29, 837, 730, 911, 493, 715, 523, 471, 748, 781, 647, 218, 651, 247, 892, 860, 16, 643, 310, 750, 346, 345, 899, 567, 463, 199, 775, 42, 907, 634, 770, 746, 670, 552, 14, 896, 211, 676, 367, 902, 726, 385, 5, 915, 334, 935, 474, 439, 320, 262, 648, 503, 923, 929, 266, 678, 187, 443, 838, 941, 294, 371, 909, 327, 488, 19, 355, 280, 336, 40, 975, 382, 514, 339, 264, 936, 428, 256, 617, 240, 650, 962, 800, 713, 475, 484, 59, 370, 314, 36, 587, 102, 791, 640, 556, 843, 125, 92, 945, 813, 415, 501, 325, 712, 342, 113, 776, 849, 87, 920, 994, 224, 307, 549, 480, 423, 328, 947, 528, 285, 841, 393, 951, 858, 531, 311, 147, 967, 115, 37, 535, 492, 300, 598, 254, 656, 237, 591, 720, 79, 23, 815, 485, 210, 197, 13, 880, 827, 434, 35, 169, 863, 369, 790, 108, 816, 442, 441, 112, 437, 972, 419, 460, 238, 179, 477, 432, 931, 550, 701, 649, 948, 894, 295, 823, 732, 175, 699, 188, 132, 344, 925, 246, 986, 525, 81, 714, 840, 966, 265, 216, 57, 160, 867, 612, 27, 76, 885, 886, 138, 875, 878, 181, 470, 99, 895, 91, 622, 628, 472, 499, 60, 420, 979, 985, 163, 521, 398, 482, 603, 520, 317, 532, 454, 71, 646, 318, 384, 487, 704, 773, 917, 365, 142, 530, 545, 879, 144, 912, 851, 759, 807, 828, 245, 526, 458, 232, 855, 222, 94, 908, 394, 767, 151, 817, 89, 287, 357, 777, 330, 18, 483, 51, 921, 582, 631, 664, 517, 274, 252, 995, 901, 171, 186, 717, 727, 478, 277, 905, 83, 293, 519, 834, 228, 120, 276, 444, 674, 534, 11, 734, 386, 259, 711, 86, 573, 324, 490, 725, 645, 326, 116, 374, 964, 831, 599, 383, 789, 54, 229, 182, 233, 258, 363, 302, 459, 505, 952, 290, 356, 360, 253, 373, 702, 615, 988, 571, 755, 970, 973, 418, 159, 990, 516, 684, 231, 818, 733, 706, 744, 502, 359, 450, 448, 97, 623, 616, 288, 322, 72, 771, 61, 399, 926, 579, 836, 976, 49, 991, 536, 864, 283, 884, 930, 396, 25, 848, 255, 433, 710, 578, 491, 758, 417, 193, 215, 870, 261, 457, 126, 391, 784, 46, 762, 141, 769, 928, 220, 792, 260, 682, 44, 806, 919, 315, 812, 6, 209, 409, 581, 303, 740, 865, 822, 963, 729, 802, 856, 111, 387, 723, 340, 933, 728, 90, 997, 563, 764, 969, 73, 635, 922, 847, 539, 202, 960, 605, 217, 379, 435, 436, 957, 551, 178, 52, 971, 685, 515, 332, 566, 7, 68, 510, 618, 412, 421, 431, 299, 620, 877, 981, 540, 106, 757, 281, 810, 114, 559, 889, 177, 799, 292, 405, 43, 155, 736, 760, 943, 555, 139, 667, 98, 234, 669, 397, 447, 481, 756, 455, 626, 548, 196, 625, 866, 62, 372, 850, 569, 416, 553, 350, 989, 301, 93, 250, 321, 660, 596, 395, 77, 174, 814, 150, 541, 611, 122, 107, 959, 347, 430, 198, 876, 70, 378, 805, 942, 692, 239, 982, 145, 50]
# [0.4302101095577636, 0.4302110921242101, 0.4302126412764012, 0.43021670228641773, 0.4302175290755098,
[338, 67, 700, 878, 12, 527, 600, 893, 897, 51, 609, 794, 148, 492, 937, 586, 885, 311, 104, 859, 236, 756, 659, 304, 582, 714, 411, 796, 212, 265, 944, 834, 101, 978, 358, 592, 983, 280, 153, 171, 161, 490, 722, 15, 665, 833, 29, 936, 601, 587, 36, 9, 999, 930, 75, 639, 139, 493, 189, 651, 724, 238, 82, 705, 506, 780, 105, 572, 906, 479, 982, 625, 392, 892, 725, 135, 972, 52, 58, 39, 645, 691, 406, 193, 456, 84, 334, 844, 573, 954, 510, 607, 674, 924, 362, 385, 635, 654, 629, 881, 664, 310, 262, 256, 356, 207, 180, 504, 474, 261, 124, 472, 900, 660, 147, 412, 69, 377, 652, 63, 623, 967, 167, 491, 257, 331, 668, 17, 316, 160, 330, 190, 154, 736, 782, 342, 364, 778, 523, 807, 214, 344, 360, 634, 539, 840, 59, 230, 952, 260, 687, 679, 290, 868, 40, 235, 537, 694, 252, 494, 986, 155, 123, 119, 195, 476, 341, 822, 643, 823, 42, 199, 835, 239, 843, 277, 721, 448, 297, 742, 373, 984, 913, 85, 898, 93, 404, 581, 302, 774, 735, 110, 87, 446, 438, 137, 141, 284, 22, 684, 35, 200, 940, 473, 237, 322, 707, 384, 346, 767, 829, 55, 908, 563, 53, 74, 170, 5, 511, 615, 115, 922, 622, 447, 670, 715, 339, 751, 401, 846, 763, 612, 327, 567, 535, 636, 928, 191, 353, 121, 734, 442, 374, 650, 588, 716, 395, 471, 184, 887, 397, 268, 247, 279, 113, 444, 804, 405, 245, 403, 576, 968, 97, 597, 367, 808, 475, 830, 852, 27, 398, 272, 315, 26, 117, 841, 34, 99, 484, 558, 56, 267, 698, 726, 836, 495, 854, 449, 222, 477, 785, 697, 749, 732, 64, 853, 580, 291, 244, 880, 350, 524, 896, 441, 465, 0, 699, 372, 644, 453, 140, 419, 640, 312, 744, 642, 309, 263, 759, 437, 522, 910, 779, 348, 813, 156, 613, 917, 549, 995, 80, 871, 747, 512, 534, 413, 133, 533, 210, 966, 332, 100, 96, 761, 584, 79, 565, 872, 509, 324, 44, 220, 498, 585, 977, 904, 98, 855, 387, 370, 864, 528, 909, 879, 503, 753, 980, 489, 706, 914, 271, 953, 981, 626, 305, 890, 951, 209, 907, 987, 663, 870, 513, 915, 215, 547, 349, 765, 923, 250, 655, 793, 766, 500, 273, 620, 961, 764, 616, 216, 432, 423, 630, 168, 108, 194, 800, 710, 861, 467, 203, 974, 445, 224, 485, 340, 741, 293, 677, 496, 637, 956, 902, 120, 740, 969, 179, 355, 554, 389, 555, 617, 433, 733, 693, 927, 352, 418, 242, 88, 501, 488, 428, 776, 820, 526, 380, 569, 30, 801, 757, 545, 276, 943, 518, 671, 935, 646, 519, 249, 205, 146, 614, 172, 188, 258, 631, 178, 111, 181, 590, 882, 683, 717, 708, 738, 152, 599, 560, 343, 164, 797, 225, 248, 536, 720, 112, 695, 296, 460, 68, 905, 777, 114, 481, 301, 275, 544, 932, 41, 462, 335, 737, 19, 440, 486, 850, 970, 144, 826, 382, 508, 369, 368, 719, 162, 420, 452, 157, 583, 556, 38, 729, 201, 1, 106, 320, 912, 916, 516, 425, 949, 873, 176, 552, 602, 386, 550, 992, 289, 129, 185, 407, 529, 945, 464, 575, 606, 960, 648, 166, 595, 169, 561, 934, 889, 628, 994, 858, 122, 429, 234, 354, 94, 838, 443, 246, 946, 345, 938, 874, 232, 875, 487, 231, 49, 791, 755, 62, 223, 811, 790, 183, 895, 656, 371, 675, 142, 422, 997, 627, 213, 760, 965, 435, 579, 514, 270, 799, 126, 918, 11, 964, 666, 845, 328, 18, 103, 383, 243, 564, 3, 365, 259, 57, 894, 317, 605, 958, 24, 300, 287, 657, 134, 709, 71, 388, 532, 812, 182, 173, 308, 379, 357, 361, 394, 920, 866, 939, 993, 197, 16, 658, 806, 598, 814, 145, 869, 251, 857, 198, 266, 703, 713, 90, 578, 689, 314, 37, 798, 482, 363, 461, 926, 682, 282, 47, 955, 470, 884, 680, 295, 883, 848, 681, 867, 831, 130, 116, 499, 459, 557, 672, 208, 570, 427, 690, 568, 899, 72, 876, 517, 151, 274, 608, 802, 20, 478, 686, 696, 158, 303, 787, 424, 704, 842, 255, 434, 333, 468, 415, 43, 957, 23, 285, 860, 525, 828, 241, 430, 421, 396, 132, 803, 632, 390, 589, 366, 46, 553, 865, 851, 240, 229, 621, 971, 329, 809, 610, 118, 772, 758, 688, 771, 318, 788, 574, 998, 150, 264, 754, 8, 321, 990, 107, 431, 159, 543, 325, 25, 531, 542, 901, 95, 454, 163, 14, 218, 149, 375, 792, 933, 662, 92, 86, 633, 351, 463, 128, 187, 450, 228, 227, 61, 306, 849, 45, 619, 743, 789, 393, 483, 409, 816, 186, 337, 347, 810, 211, 647, 947, 131, 269, 286, 125, 783, 702, 507, 177, 21, 752, 175, 775, 288, 408, 336, 31, 773, 669, 359, 825, 959, 78, 653, 81, 77, 551, 723, 562, 618, 165, 313, 931, 502, 505, 254, 962, 596, 530, 298, 676, 624, 817, 410, 819, 28, 577, 649, 7, 929, 89, 48, 414, 827, 376, 2, 604, 83, 979, 638, 292, 217, 204, 996, 673, 591, 54, 4, 378, 711, 319, 769, 458, 73, 821, 786, 950, 417, 455, 832, 402, 641, 466, 13, 712, 294, 837, 299, 219, 685, 202, 571, 540, 221, 566, 948, 391, 863, 66, 727, 480, 416, 976, 942, 768, 226, 985, 323, 903, 206, 10, 136, 521, 541, 839, 32, 326, 891, 973, 381, 76, 283, 515, 439, 60, 678, 805, 667, 925, 143, 877, 824, 538, 138, 520, 102, 469, 911, 233, 593, 718, 603, 451, 281, 989, 750, 174, 856, 426, 730, 748, 784, 307, 399, 196, 497, 746, 921, 739, 594, 795, 559, 888, 548, 436, 253, 975, 963, 6, 815, 847, 278, 65, 701, 728, 770, 862, 731, 941, 70, 109, 661, 781, 127, 457, 33, 745, 762, 919, 192, 611, 886, 991, 692, 91, 400, 546, 818, 988, 50]


# 700, 12,